In [1]:
#==================================================================
#Program: DEVEL_3
#Version: 1.0
#Author: David Helminiak
#Date Created: September 5, 2024
#Date Last Modified: September 22, 2024
#Description: Re-implementation of a unified block classifier code for breast cancer data
#Operation: Move back into main program directory before running.
#Status: Deprecated - Completed and transitioned into .py file structure
#==================================================================

#Have the notebook fill more of the display width
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:80% !important; }</style>"))

#Items otherwise covered when not running this code in a notebook
import tempfile
dir_tmp = tempfile.TemporaryDirectory(prefix='TMP_').name
configFileName = './CONFIG_0-TEST'

In [3]:
#==================================================================
#CONFIG PARAMETERS
#==================================================================

#TASKS
#==================================================================
#Should classification/cross-validation be performed on blocks and the portions of WSI they were extracted from 
classifierBlocks = True

#Should classifier model components be saved/exported
classifierExport = True

#Should whole WSI be classified and evaluated
classifierWSI = True

#Should training data be generated for reconstruction model
classifierRecon = True

#==================================================================


#COMPUTE
#==================================================================
#How many samples should be submitted in a batch through pytorch models used in classifier
#Incrementing in powers of 2 recommended to best leverage common GPU hardware designs
#For ResNet and DenseNet a 2080TI 11GB can handle 64x3x224x224 (resizeSize=224) or 16x3x400x400 (resizeSize=0)
#In some cases, the GPU memory may not be fully released (even when explicitly told to do so), so using a lower batch size may help prevent OOM
batchsizeClassifier = 32

#==================================================================


#CLASSIFICATION - BLOCKS
#==================================================================
#Should features be extracted for blocks and overwrite previously generated files
overwrite_blocks_features = True

#Should saliency maps be determined for blocks and overwrite previously generated files
overwrite_blocks_saliencyMaps = True

#Should the decision fusion mode be used for block classification (default: True)
fusionMode_blocks = True

#Should saliency maps and their overlays be visualized for block WSI
visualizeSaliencyMaps_blocks = True

#Should label grids and their overlays be visualized for block WSI; will overwrite previously generated files
#Files should be updated if 1) thresholdWSI_GT is changed or 2) thresholdWSI is changed and thresholdWSI_GT is True
visualizeLabelGrids_blocks = True

#Should prediction grids and their overlays be visualized for block WSI
visualizePredictionGrids_blocks = True

#What ratio of malignant to benign blocks should be used to label a whole WSI as malignant (default: 0.15)
#Unknown what the original work used for this value, but chose a value (0.15 - being in range of 0.12-0.19) that can replicate results from original work
#If this value is changed and thresholdWSI_GT is True, then should enable visualizeLabelGrids_blocks to update stored data
thresholdWSI = 0.15

#Should the thresholdWSI be used to determine the ground-truth label (True) or use recorded metadata (False) (default: False)
#Unsure exactly what the original work did, but suspect it may have used method equivalent of 'True', given its usage replicates the original results.
#Strongly recommend using recorded metadata! The option has been left to enable replication of original results.
#If this value is changed, then should enable visualizeLabelGrids_blocks to update stored data
thresholdWSI_GT = False

#If folds for XGB classifier cross validation should be manually defined (e.g. [['S1', 'S3'], ['S4', 'S2']]), else use specify number of folds to generate
#Default matches folds used in prior work (https://doi.org/10.3389/fonc.2023.1179025)
#Omits 6 available samples, with folds holding: 11, 12, 12, 12, 13 samples respectively; this may have been to better balance class distribution
#Presently, all available (non-excluded) samples (not just those in manualFolds) are currently used for training the exported/utilized final classifier
manualFolds = [['2_1', '9_3', '11_3', '16_3', '34_1', '36_2', '40_2', '54_2', '57_2', '60_1', '62_1'],
               ['17_5', '20_3', '23_3', '24_2', '28_2', '30_2', '33_3', '51_2', '52_2', '59_2', '63_3', '66_2'], 
               ['12_1', '14_2', '22_3', '26_3', '35_4', '44_1', '45_1', '47_2', '49_1', '53_2', '56_2', '68_1'], 
               ['4_4', '5_3', '8_1', '10_3', '25_3', '27_1', '29_2', '37_1', '42_3', '48_3', '50_1', '69_1'], 
               ['7_2', '15_4', '19_2', '31_1', '43_1', '46_2', '55_2', '58_2', '61_1', '64_1', '65_1', '67_1', '70_1']]

#Which pre-trained weight sets should be used for ResNet and DenseNet model: 'IMAGENET1K_V1', 'IMAGENET1K_V2'
#Unclear which weights were used for TensorFlow ResNet50 variant in original implementation, but V2 did improve scores a bit when using resizeSize = 0
weightsResNet = 'IMAGENET1K_V2'

#Which pre-trained weight sets should be used for DenseNet model: 'IMAGENET1K_V1'
weightsDenseNet = 'IMAGENET1K_V1'

#==================================================================


#CLASSIFICATION - WSI
#==================================================================
#Should WSI preparation and block extraction overwrite previously generated files)
overwrite_WSI_blocks = True

#Should features be extracted for WSI extracted blocks and overwrite previously generated files
overwrite_WSI_features = True

#Should saliency maps be determined for WSI extracted blocks and overwrite previously generated files
overwrite_WSI_saliencyMaps = True

#Should the decision fusion mode be used for block classification (default: True)
fusionMode_WSI = True

#Should saliency maps and their overlays be visualized for WSI
visualizeSaliencyMaps_WSI = True

#Should prediction grids and their overlays be visualized for WSI
visualizePredictionGrids_WSI = True

#==================================================================

#CLASSIFICATION - Reconstruction
#==================================================================
#Should visuals of the reconstruction model input data be generated
visualizeInputData_recon = True

#==================================================================

#RARELY CHANGED
#==================================================================

#What is the camera resolution in mm/pixel for the instrument that acquired the data being used
cameraResolution = 0.00454

#What is the minimum area/quantity (in mm^2) of foreground data that should qualify a block for classification
#Decrease for increased sensitivity and vice versa; result should not exceed blockSize*cameraResolution
#As the classifier was not trained to handle blank background blocks, setting to low will harm performance
minimumForegroundArea = 1.0**2

#Minimum value [0, 255] for a grayscale pixel to be considered as a foreground location during block extraction (default: 11)
#-1 will automatically determine a new value as the minimum Otsu threshold across all available WSI; default value from prior determination
blockBackgroundValue = -1

#When splitting WSI images, what size should the resulting blocks be (default: 400)
#Should remain consistent with block sizes given for training
blockSize = 400

#Specify what symmetrical dimension blocks should be resized to; if no resizing is desired leave as 0 (default: 224)
#Leaving as 0 will increase training time (also must change batchsizeClassifier), but can lead to improved scores
#Original implementation uses 224, though with adaptive average pooling this isn't actually neccessary
resizeSize_blocks = 224

#Specify what symmetrical dimension WSI should be resized to when generating saliency maps (default: 224)
#If fusion method were to be further developed, this should be changed to maintain the original sample aspect ratio. 
resizeSize_WSI = 224

#How thick should the grid lines be when generating overlay images (defualt: 50)
gridThickness = 50

#Should saliency map overlays be placed over data converted to grayscale for clearer visualization (default: True)
overlayGray = True

#Should images with overlay data (and grid maps) be saved to lossless (.tif) or compressed (.jpg) image format (default: False)
overlayLossless = False

#For .jpg image outputs, what should the compression quality (%) be (default: 95)
#WARNING: Setting to 100 is not sufficient to generate in lossless/exact outputs; if that is desired, use overlayLossless instead! 
exportQuality = 95

#What weight should be used when overlaying data
overlayWeight = 0.5

#Define labels used for normal/benign tissue
#'a': normal adipose.
#'s': normal stroma tissue excluding adipose.
#'o': other normal tissue including parenchyma, adenosis, lobules, blood vessels, etc.
labelsBenign = ['a', 's', 'o', 'normal']

#Define labels used for malignant tissue
#'d': IDC tumor
#'l': ILC tumor
#'ot': other tumor areas including DCIS, biopsy site, and slightly defocused tumor regions.
labelsMalignant = ['d', 'l', 'ot', 'tumor']

#Define labels used for tissues to be excluded
#'ft': defocused but still visually tumor-like areas.
#'f': severly out-of-focusing areas. 
#'b': background. 
#'e': bubbles.
labelsExclude = ['ft', 'f', 'b', 'e', 'exclude']

#==================================================================



#Already in setup CONFIG (avoid duplication during copy)
#==================================================================
#Should parallelization calls be used to leverage multithreading where able
parallelization = True

#If parallelization is enabled, how many CPU threads should be used in Ray tasks? (0 will use any/all available)
#Recommend starting at half of the available system threads if using hyperthreading,
#or 1-2 less than the number of system CPU cores if not using hyperthreading.
#Adjust to where the CPU just below 100% usage during parallel operations 
availableThreads = 16

#Which GPU(s) devices should be used (last specified used for data processing and model training); (default: [-1], any/all available; CPU only: [])
gpus = [-1]

#Should training/validation data be entirely stored on GPU (default: True; improves training/validation efficiency, set to False if OOM occurs)
storeOnDevice = True
   
#RNG seed value to ensure run-to-run consistency (-1 to disable)
manualSeedValue = 0

#Should warnings and info messages be shown during operation
debugMode = True

#Should progress bars be visualized with ascii characters
asciiFlag = False

#==================================================================


In [4]:
#==================================================================
#MAIN - Run codes that have already been completed
#==================================================================

exec(open("./CODE/EXTERNAL.py", encoding='utf-8').read())

exec(open("./CODE/COMPUTE.py", encoding='utf-8').read())


In [5]:
#==================================================================
#MODEL_CLASS
#==================================================================

#Load and preprocess data image files
#ToTensor() swaps axes (so not used); convert to contiguous torch tensor manually
#Rescaling and changing data type only after resizing (otherwise can escape [0, 1])
class DataPreprocessing_Classifier(Dataset):
    def __init__(self, filenames, resizeSize):
        super().__init__()
        self.filenames = filenames
        self.numFiles = len(self.filenames)
        if resizeSize > 0: self.transform = generateTransform([resizeSize, resizeSize], True, True)
        else: self.transform = generateTransform([], True, True)
        
    #Rearranging import dimensions, allows resize transform before tensor-conversion/rescaling; preserves precision and output data range
    def __getitem__(self, index): return self.transform(np.moveaxis(cv2.cvtColor(cv2.imread(self.filenames[index], cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB), -1, 0))
    
    def __len__(self): return self.numFiles
    
#Load/synchronize data labeling, drop excluded rows, and extract relevant metadata
def loadMetadata_blocks(filename):
    metadata = pd.read_csv(filename, header=0, names=['Sample', 'Index', 'Row', 'Column', 'Label'], converters={'Sample':str,'Index':str, 'Row':int, 'Column':int, 'Label':str})
    metadata['Label'] = metadata['Label'].replace(labelsBenign, labelBenign)
    metadata['Label'] = metadata['Label'].replace(labelsMalignant, labelMalignant)
    metadata['Label'] = metadata['Label'].replace(labelsExclude, labelExclude)
    metadata = metadata.loc[metadata['Label'] != labelExclude]
    return [np.squeeze(data) for data in np.split(np.asarray(metadata), [1, 2, 4], -1)]

#Load/synchronize data labeling, drop excluded rows, and extract relevant metadata
def loadMetadata_WSI(filename):
    metadata = pd.read_csv(filename, header=0, names=['Sample', 'Label'], converters={'Sample':str, 'Label':str})
    metadata['Label'] = metadata['Label'].replace(labelsBenign, labelBenign)
    metadata['Label'] = metadata['Label'].replace(labelsMalignant, labelMalignant)
    metadata['Label'] = metadata['Label'].replace(labelsExclude, labelExclude)
    metadata = metadata.loc[metadata['Label'] != labelExclude]
    return [np.squeeze(data) for data in np.split(np.asarray(metadata), 2, -1)]

#Extract blocks and determine associated metadata for referenced WSI files; abstraction allows for isolation of WSI data used for training the classifier
def extractBlocks(WSIFilenames):
    
    #Extract uniform, non-overlapping blocks from each WSI; may be too memory intensive and RW-bottlenecked to parallelize efficiently
    blockNames, blockFilenames, blockSampleNames, blockLocations, cropData, paddingData, shapeData = [], [], [], [], [], [], []
    for filename in tqdm(WSIFilenames, desc='Block Extraction', leave=True, ascii=asciiFlag):
        
        #Extract base sample name
        sampleName = os.path.basename(filename).split('.jpg')[0]
        
        #Load WSI image
        imageWSI = cv2.cvtColor(cv2.imread(filename, cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB)
        
        #Tried a few quicker methods for pre-processing before applying some foreground segmentation
        #No tangible benefits were observed in performance, but leaving in code at least once for archival reference
        
        #Histogram equalization
        #y, cr, cb = cv2.split(cv2.cvtColor(imageWSI, cv2.COLOR_RGB2YCrCb))
        #imageMask = cv2.cvtColor(cv2.merge((cv2.equalizeHist(y), cr, cb)), cv2.COLOR_YCR_CB2RGB)
        
        #Denoising
        #imageMask = cv2.fastNlMeansDenoising(cv2.cvtColor(imageWSI, cv2.COLOR_RGB2GRAY), None, h=3, templateWindowSize=7, searchWindowSize=21)
        
        #CLAHE
        #imageMask = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8)).apply(cv2.cvtColor(imageWSI, cv2.COLOR_RGB2GRAY))
        
        #Isolate only the largest non-zero area; would not be able to handle samples with disconnected tissue segments
        #imageMask = cv2.threshold(cv2.cvtColor(imageWSI, cv2.COLOR_RGB2GRAY),0,255,cv2.THRESH_BINARY)[1]
        #imageMask = max(cv2.findContours(imageMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0], key=cv2.contourArea)
        
        #Crop WSI to the foreground area using Otsu
        imageWSI_gray = cv2.cvtColor(imageWSI, cv2.COLOR_RGB2GRAY)
        x, y, w, h = cv2.boundingRect(cv2.threshold(imageWSI_gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]) 
        imageWSI = imageWSI[y:y+h, x:x+w]
        imageWSI_gray = imageWSI_gray[y:y+h, x:x+w]
        cropData.append([y, y+h, x, x+w])
        
        #Pad the image as needed (as symmetrially as possible) for an even division by the specified block size; compute numBlocks per row/column
        padHeight = (int(np.ceil(imageWSI.shape[0]/blockSize))*blockSize)-imageWSI.shape[0]
        padWidth = (int(np.ceil(imageWSI.shape[1]/blockSize))*blockSize)-imageWSI.shape[1]
        padTop, padLeft = padHeight//2, padWidth//2
        padBottom, padRight = padTop+(padHeight%2), padLeft+(padWidth%2)
        imageWSI = np.pad(imageWSI, ((padTop, padBottom), (padLeft, padRight), (0, 0)))
        imageWSI_gray = np.pad(imageWSI_gray, ((padTop, padBottom), (padLeft, padRight)))
        paddingData.append([padTop, padBottom, padLeft, padRight])
        numBlocksRow, numBlocksCol = imageWSI.shape[0]//blockSize, imageWSI.shape[1]//blockSize
        shapeData.append([numBlocksRow, numBlocksCol])
        
        #Split the WSI (color and grayscale) into blocks and flatten
        imageWSI = imageWSI.reshape(numBlocksRow, blockSize, numBlocksCol, blockSize, imageWSI.shape[2]).swapaxes(1,2)
        imageWSI = imageWSI.reshape(-1, imageWSI.shape[2], imageWSI.shape[3], imageWSI.shape[4])
        imageWSI_gray = imageWSI_gray.reshape(numBlocksRow, blockSize, numBlocksCol, blockSize).swapaxes(1,2)
        imageWSI_gray = imageWSI_gray.reshape(-1, imageWSI_gray.shape[2], imageWSI_gray.shape[3])
        
        #Setup directory to store blocks
        dir_WSI_sampleBlocks = dir_WSI_blocks + 'S' + sampleName + os.path.sep
        if not os.path.exists(dir_WSI_sampleBlocks): os.makedirs(dir_WSI_sampleBlocks)
        
        #Record metadata for each block that has a specified percentage of foreground data and save each to disk
        blockIndex = 0
        for rowNum in range(0, numBlocksRow):
            for colNum in range(0, numBlocksCol):
                if np.mean(imageWSI_gray[blockIndex] >= blockBackgroundValue) >= blockBackgroundRatio: 
                    locationRow, locationColumn= rowNum*blockSize, colNum*blockSize
                    blockLocations.append([locationRow, locationColumn])
                    blockName = sampleName+'_'+str(blockIndex)+'_'+str(locationRow)+'_'+str(locationColumn)
                    blockNames.append(blockName)
                    blockSampleNames.append(sampleName)
                    blockFilenames.append(dir_WSI_sampleBlocks+'PS'+blockName+'.tif')
                    exportImage(blockFilenames[-1], imageWSI[blockIndex])
                blockIndex += 1
    
    return np.asarray(blockNames), np.asarray(blockFilenames), np.asarray(blockSampleNames), np.asarray(blockLocations), np.asarray(cropData), np.asarray(paddingData), np.asarray(shapeData)

#Compute metrics for a classification result and visualize/save them as needed
def computeClassificationMetrics(labels, predictions, baseFilename):
    
    #Specify data class labels
    displayLabels = ['Benign', 'Malignant']
    classLabels = np.arange(0, len(displayLabels), 1)
    
    #Generate/store a classification report: precision, recall, f1-score, and support
    classificationReport = classification_report(labels, predictions, labels=classLabels, target_names=displayLabels, output_dict=True)#, zero_division=0.0)
    classificationReport = pd.DataFrame(classificationReport).transpose()
    classificationReport.to_csv(baseFilename + '_classificationReport.csv')

    #Generate a confusion matrix and extract relevant statistics
    confusionMatrix = confusion_matrix(labels, predictions, labels=classLabels)
    tn, fp, fn, tp = confusionMatrix.ravel()
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    
    #Store relevant statistics
    summaryReport = {'Accuracy': accuracy, 'Sensitivity': sensitivity, 'Specificity': specificity}
    summaryReport = pd.DataFrame.from_dict(summaryReport, orient='index')
    summaryReport.to_csv(baseFilename + '_summaryReport.csv')
    
    #Store confusion matrix
    displayCM = ConfusionMatrixDisplay(confusionMatrix, display_labels=displayLabels)
    displayCM.plot(cmap='Blues')
    plt.tight_layout()
    plt.savefig(baseFilename+'_confusionMatrix.tif')
    plt.close()
    
#Generate a torch transform needed for preprocessing image data
def generateTransform(resizeSize=[], rescale=False, normalize=False):
    transform = [lambda inputs : torch.from_numpy(inputs).contiguous()]
    if len(resizeSize) > 0: transform.append(v2.Resize(tuple(resizeSize)))
    if rescale: transform.append(lambda inputs: inputs.to(dtype=torch.get_default_dtype()).div(255))
    if normalize: transform.append(transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))
    return transforms.Compose(transform)

#Export lossless RGB image data to disk
def exportImage(filename, image):
    if filename.split('.')[-1] == 'tif': writeSuccess = cv2.imwrite(filename, cv2.cvtColor(image, cv2.COLOR_RGB2BGR), params=(cv2.IMWRITE_TIFF_COMPRESSION, 1))
    elif filename.split('.')[-1] == 'jpg': writeSuccess = cv2.imwrite(filename, cv2.cvtColor(image, cv2.COLOR_RGB2BGR), [int(cv2.IMWRITE_JPEG_QUALITY), exportQuality])
    else: sys.exit('\nError - Specified image output format has not been implemented.')
    if not writeSuccess: sys.exit('\nError - Unable to write file: ', filename)

#OpenCV does not output sharp corners with its rectangle method...unless it's filled in
def rectangle(image, startPos, endPos, color):
    image = cv2.rectangle(image, startPos, endPos, color, -1)
    image = cv2.rectangle(image, (startPos[0]+gridThicknessOffset, startPos[1]+gridThicknessOffset), (endPos[0]-gridThicknessOffset, endPos[1]-gridThicknessOffset), (0, 0, 0), -1)
    return image


In [6]:
#==================================================================
#INTERNAL_DIRECTORY
#==================================================================

#Indicate and setup the destination folder for results of this configuration
destResultsFolder = './RESULTS_'+os.path.splitext(os.path.basename(configFileName).split('_')[1])[0]

#If the folder already exists, either remove it, or append a novel value to it
if os.path.exists(destResultsFolder):
    if not preventResultsOverwrite: 
        shutil.rmtree(destResultsFolder)
    else: 
        destinationNameValue = 0
        destResultsFolder_Base = copy.deepcopy(destResultsFolder)
        while True:
            destResultsFolder = destResultsFolder_Base + '_' + str(destinationNameValue)
            if not os.path.exists(destResultsFolder): break
            destinationNameValue += 1

#Data input directories and file locations
#=============================================================================
#Global
dir_data = '.' + os.path.sep + 'DATA' + os.path.sep
dir_results = '.' + os.path.sep + 'RESULTS' + os.path.sep
dir_classifier_models = dir_results + 'MODELS' + os.path.sep

#Block classification
dir_blocks_data = dir_data + 'BLOCKS' + os.path.sep
dir_blocks_inputBlocks = dir_blocks_data + 'INPUT_BLOCKS' + os.path.sep
file_blocks_metadataBlocks = dir_blocks_inputBlocks + 'metadata_blocks.csv'
dir_blocks_inputWSI = dir_blocks_data + 'INPUT_WSI' + os.path.sep
file_blocks_metadataWSI = dir_blocks_inputWSI + 'metadata_WSI.csv'

dir_blocks_features = dir_blocks_data + 'OUTPUT_FEATURES' + os.path.sep
dir_blocks_salicencyMaps = dir_blocks_data + 'OUTPUT_SALIENCY_MAPS' + os.path.sep

dir_blocks_visuals = dir_blocks_data + 'OUTPUT_VISUALS' + os.path.sep
dir_blocks_visuals_saliencyMaps = dir_blocks_visuals + 'SALIENCY_MAPS' + os.path.sep
dir_blocks_visuals_overlaidSaliencyMaps = dir_blocks_visuals + 'OVERLAID_SALICENCY_MAPS' + os.path.sep
dir_blocks_visuals_labelGrids = dir_blocks_visuals + 'LABEL_GRIDS' + os.path.sep
dir_blocks_visuals_overlaidLabelGrids = dir_blocks_visuals + 'OVERLAID_LABEL_GRIDS' + os.path.sep

dir_blocks_results = dir_results + 'BLOCKS' + os.path.sep
dir_blocks_results_labelGrids = dir_blocks_results + 'LABEL_GRIDS' + os.path.sep
dir_blocks_results_overlaidLabelGrids = dir_blocks_results + 'OVERLAID_LABEL_GRIDS' + os.path.sep


#WSI classification
dir_WSI_data = dir_data + 'WSI' + os.path.sep
dir_WSI_inputs = dir_WSI_data + 'INPUT_WSI' + os.path.sep
file_WSI_metadataWSI = dir_WSI_inputs + 'metadata_WSI.csv'

dir_WSI_blocks = dir_WSI_data + 'OUTPUT_BLOCKS' + os.path.sep
dir_WSI_features = dir_WSI_data + 'OUTPUT_FEATURES' + os.path.sep
dir_WSI_saliencyMaps = dir_WSI_data + 'OUTPUT_SALIENCY_MAPS' + os.path.sep

dir_WSI_visuals = dir_WSI_data + 'OUTPUT_VISUALS' + os.path.sep
dir_WSI_visuals_saliencyMaps = dir_WSI_visuals + 'SALIENCY_MAPS' + os.path.sep
dir_WSI_visuals_overlaidSaliencyMaps = dir_WSI_visuals + 'OVERLAID_SALIENCY_MAPS' + os.path.sep

dir_WSI_results = dir_results + 'WSI' + os.path.sep
dir_WSI_results_labelGrids = dir_WSI_results + 'LABEL_GRIDS' + os.path.sep
dir_WSI_results_overlaidLabelGrids = dir_WSI_results + 'OVERLAID_LABEL_GRIDS' + os.path.sep

#Reconstruction model input data classification
dir_recon_data = dir_data + 'RECON' + os.path.sep
dir_recon_inputData = dir_recon_data + 'INPUT_RECON' + os.path.sep
dir_recon_visuals_inputData = dir_recon_data + 'INPUT_VISUALS' + os.path.sep

dir_recon_results = dir_results + 'RECON' + os.path.sep
dir_recon_results_train = dir_recon_results + 'TRAIN' + os.path.sep
dir_recon_results_test = dir_recon_results + 'TEST' + os.path.sep

#=============================================================================


#If folders do not exist, but their use is enabled, exit the program
#=============================================================================
#Block classification
if not os.path.exists(dir_data): sys.exit('\nError - Required folder: ' + dir_data + ' does not exist.')
if not os.path.exists(dir_blocks_data) and (classifierBlocks or classifierExport): sys.exit('\nError - Required folder: ' + dir_blocks_data + ' does not exist.')
if not os.path.exists(dir_blocks_inputBlocks) and (classifierBlocks or classifierExport): sys.exit('\nError - Required folder: ' + dir_blocks_inputBlocks + ' does not exist.')
if not os.path.exists(dir_blocks_inputWSI) and (classifierBlocks or classifierExport): sys.exit('\nError - Required folder: ' + dir_blocks_inputWSI + ' does not exist.')

#WSI classification
if not os.path.exists(dir_WSI_data) and classifierWSI: sys.exit('\nError - Required folder: ' + dir_WSI_data + ' does not exist.')
if not os.path.exists(dir_WSI_inputs) and classifierWSI: sys.exit('\nError - Required folder: ' + dir_WSI_inputs + ' does not exist.')

#Reconstruction model input data classification
if (len(glob.glob(dir_WSI_features+'*.npy'))!=2 or len(glob.glob(dir_WSI_saliencyMaps+'*.npy'))!=2) and (classifierRecon and not classifierWSI): sys.exit('\Error - WSI block features/weights, required for generation of reconstruction model input data are not available.')

#=============================================================================
#If a task is diabled, then overwrites should be disabled to prevent overwrite of existing data
#If for a given task, file overwrites are not enabled, and the needed files do not exist, then enable the relevant overwrite(s)
#=============================================================================

if (classifierBlocks or classifierExport):
    if not overwrite_blocks_features and len(glob.glob(dir_blocks_features+'*.npy'))==0: overwrite_blocks_features = True    

if classifierBlocks:
    if not overwrite_blocks_saliencyMaps and fusionMode_blocks and len(glob.glob(dir_blocks_salicencyMaps+'*.npy'))==0: overwrite_blocks_saliencyMaps = True
else:
    overwrite_blocks_features = False
    overwrite_blocks_saliencyMaps = False

if classifierWSI:
    if not overwrite_WSI_blocks and len(glob.glob(dir_WSI_blocks+'*.npy'))==0: overwrite_WSI_blocks = True
    if not overwrite_WSI_features and len(glob.glob(dir_WSI_features+'*.npy'))==0: overwrite_WSI_features = True
    if not overwrite_WSI_saliencyMaps and fusionMode_WSI and len(glob.glob(dir_WSI_saliencyMaps+'*.npy'))==0: overwrite_WSI_saliencyMaps = True
else:
    overwrite_WSI_blocks = False
    overwrite_WSI_features = False
    overwrite_WSI_saliencyMaps = False
    
#=============================================================================


#Regenerate empty files/folders that are to be overwritten
#=============================================================================
#Result folders
if not os.path.exists(dir_results): os.makedirs(dir_results)
if classifierExport and os.path.exists(dir_classifier_models): shutil.rmtree(dir_classifier_models)
if classifierBlocks and os.path.exists(dir_blocks_results): shutil.rmtree(dir_blocks_results)
if classifierWSI and os.path.exists(dir_WSI_results): shutil.rmtree(dir_WSI_results)
if classifierRecon and os.path.exists(dir_recon_results): shutil.rmtree(dir_recon_results)
checkDirectories = [dir_classifier_models,
                    dir_blocks_results,
                    dir_blocks_results_labelGrids,
                    dir_blocks_results_overlaidLabelGrids, 
                    dir_WSI_results, 
                    dir_WSI_results_labelGrids,
                    dir_WSI_results_overlaidLabelGrids,
                    dir_recon_results,
                    dir_recon_results_train,
                    dir_recon_results_test
                   ]
for directory in checkDirectories: os.makedirs(directory)
    
#Block classification
if overwrite_blocks_features and os.path.exists(dir_blocks_features): shutil.rmtree(dir_blocks_features)
if overwrite_blocks_saliencyMaps: 
    if os.path.exists(dir_blocks_salicencyMaps): shutil.rmtree(dir_blocks_salicencyMaps)
    if os.path.exists(dir_blocks_visuals_saliencyMaps): shutil.rmtree(dir_blocks_visuals_saliencyMaps)
    if os.path.exists(dir_blocks_visuals_overlaidSaliencyMaps): shutil.rmtree(dir_blocks_visuals_overlaidSaliencyMaps)
if visualizeLabelGrids_blocks:
    if os.path.exists(dir_blocks_visuals_labelGrids): shutil.rmtree(dir_blocks_visuals_labelGrids)
    if os.path.exists(dir_blocks_visuals_overlaidLabelGrids): shutil.rmtree(dir_blocks_visuals_overlaidLabelGrids)
    
#WSI classification
if overwrite_WSI_blocks and os.path.exists(dir_WSI_blocks): shutil.rmtree(dir_WSI_blocks)
if overwrite_WSI_features and os.path.exists(dir_WSI_features): shutil.rmtree(dir_WSI_features)
if overwrite_WSI_saliencyMaps: 
    if os.path.exists(dir_WSI_saliencyMaps): shutil.rmtree(dir_WSI_saliencyMaps)
    if os.path.exists(dir_WSI_visuals_saliencyMaps): shutil.rmtree(dir_WSI_visuals_saliencyMaps)
    if os.path.exists(dir_WSI_visuals_overlaidSaliencyMaps): shutil.rmtree(dir_WSI_visuals_overlaidSaliencyMaps)

#Reconstruction model input data classification
if classifierRecon:
    if os.path.exists(dir_recon_data): shutil.rmtree(dir_recon_data)
if visualizeInputData_recon:
    if os.path.exists(dir_recon_visuals_inputData): shutil.rmtree(dir_recon_visuals_inputData)

#=============================================================================


#If data output (not result) folders do not exist, then create them
#=============================================================================
#Block classification
checkDirectories = [dir_blocks_features, 
                    dir_blocks_salicencyMaps,
                    dir_blocks_visuals,
                    dir_blocks_visuals_saliencyMaps,
                    dir_blocks_visuals_overlaidSaliencyMaps,
                    dir_blocks_visuals_labelGrids, 
                    dir_blocks_visuals_overlaidLabelGrids
                   ]
for directory in checkDirectories: 
    if not os.path.exists(directory): os.makedirs(directory)

#WSI classification
checkDirectories = [dir_WSI_blocks, 
                    dir_WSI_features,
                    dir_WSI_saliencyMaps,
                    dir_WSI_visuals, 
                    dir_WSI_visuals_saliencyMaps,
                    dir_WSI_visuals_overlaidSaliencyMaps,
                   ]
for directory in checkDirectories: 
    if not os.path.exists(directory): os.makedirs(directory)
        
#Reconstruction model input data classification 
checkDirectories = [dir_recon_data, 
                    dir_recon_inputData,
                    dir_recon_visuals_inputData
                   ]
for directory in checkDirectories: 
    if not os.path.exists(directory): os.makedirs(directory)

#=============================================================================



In [7]:
#Classify image files
class Classifier():
    
    #Load blocks specified by provided filenames, extract relevant features, and setup additional model components needed for training/evaluation
    def __init__(self, dataType, blockNames, blockFilenames, blockSampleNames, blockLocations, sampleNames, WSIFilenames, WSILabels, cropData=[], paddingData=[], shapeData=[], blockLabels=None):
        
        #Store input variables internally
        self.dataType = dataType
        self.blockNames = blockNames
        self.blockFilenames = blockFilenames
        self.blockSampleNames = blockSampleNames
        self.blockLocations = blockLocations
        self.sampleNames = sampleNames
        self.WSIFilenames = WSIFilenames
        self.WSILabels = WSILabels
        self.cropData = cropData
        self.paddingData = paddingData
        self.shapeData = shapeData
        self.blockLabels = blockLabels
        
        #Specify internal object directories/data according to data type
        if dataType == 'blocks':
            self.dir_results = dir_blocks_results
            self.dir_features = dir_blocks_features
            self.dir_saliencyMaps = dir_blocks_salicencyMaps
            self.dir_visuals_saliencyMaps = dir_blocks_visuals_saliencyMaps
            self.dir_visuals_overlaidSaliencyMaps = dir_blocks_visuals_overlaidSaliencyMaps
            self.dir_visuals_labelGrids = dir_blocks_visuals_labelGrids
            self.dir_visuals_overlaidLabelGrids = dir_blocks_visuals_overlaidLabelGrids
            self.dir_results_labelGrids = dir_blocks_results_labelGrids
            self.dir_results_overlaidLabelGrids = dir_blocks_results_overlaidLabelGrids
            self.visualizeLabelGrids = visualizeLabelGrids_blocks
            self.visualizePredictionGrids = visualizePredictionGrids_blocks
        elif dataType == 'WSI' or dataType == 'recon':
            self.dir_results = dir_WSI_results
            self.dir_features = dir_WSI_features
            self.dir_saliencyMaps = dir_WSI_saliencyMaps
            self.dir_visuals_saliencyMaps = dir_WSI_visuals_saliencyMaps
            self.dir_visuals_overlaidSaliencyMaps = dir_WSI_visuals_overlaidSaliencyMaps
            self.dir_results_labelGrids = dir_WSI_results_labelGrids
            self.dir_results_overlaidLabelGrids = dir_WSI_results_overlaidLabelGrids
            self.visualizeLabelGrids = False
            self.visualizePredictionGrids = visualizePredictionGrids_WSI
        else:
            sys.error('\nError - Unknown data type used when creating classifier object.')
            
        #If reconstruction model input data is being generated, then load features and weights of applicable blocks
        if dataType == 'recon':
            blockFeatures_WSI_blocks = np.load(self.dir_features + 'blockFeatures_WSI_blocks.npy', allow_pickle=True)
            blockFeatures_WSI_WSI = np.load(self.dir_features + 'blockFeatures_WSI_WSI.npy', allow_pickle=True)
            self.blockFeatures = np.concatenate([blockFeatures_WSI_blocks, blockFeatures_WSI_WSI])
            np.save(dir_recon_inputData + 'blockFeatures', self.blockFeatures)
            del blockFeatures_WSI_WSI, blockFeatures_WSI_blocks
            if fusionMode_WSI:
                blockWeights_WSI_blocks = np.load(self.dir_saliencyMaps + 'blockWeights_WSI_blocks.npy', allow_pickle=True)
                blockWeights_WSI_WSI = np.load(self.dir_saliencyMaps + 'blockWeights_WSI_WSI.npy', allow_pickle=True)
                self.blockWeights = np.concatenate([blockWeights_WSI_blocks, blockWeights_WSI_WSI])
                np.save(dir_recon_inputData + 'blockWeights', self.blockWeights)
                del blockWeights_WSI_WSI, blockWeights_WSI_blocks
        
        #Prepare data objects for obtaining/processing PyTorch model inputs
        #Using num_workers>1 in the DataLoader objects causes bizzare semaphore/lock/descriptor issues/warnings; still appears to work, but keeping to 1 for safety
        self.device = f"cuda:{gpus[-1]}" if len(gpus) > 0 else "cpu"
        self.torchDevice = torch.device(self.device)
        self.blockData = DataPreprocessing_Classifier(blockFilenames, resizeSize_blocks)
        self.blockDataloader = DataLoader(self.blockData, batch_size=batchsizeClassifier, num_workers=1, shuffle=False, pin_memory=True)
        self.numBlockData = len(self.blockDataloader)
        self.WSIData = DataPreprocessing_Classifier(WSIFilenames, resizeSize_WSI)
        self.WSIDataloader = DataLoader(self.WSIData, batch_size=batchsizeClassifier, num_workers=1, shuffle=False, pin_memory=True)
        self.numWSIData = len(self.WSIDataloader)
        
        #Set default cuda device for XGBClassifier input data
        if len(gpus) > 0: cp.cuda.Device(gpus[-1]).use()
        
    def computeFeatures(self, overwrite_Features, suffix=''):
        
        #Extract or load features for the indicated block files
        if overwrite_Features: 
        
            #Load pretrained ResNet50 model and set to evaluation mode
            model_ResNet = models.resnet50(weights=weightsResNet).to(self.torchDevice)
            _ = model_ResNet.train(False)

            #Extract features for each batch of sample block images
            self.blockFeatures = []
            for data in tqdm(self.blockDataloader, total=self.numBlockData, desc='Feature Extraction', leave=True, ascii=asciiFlag):
                self.blockFeatures += model_ResNet(data.to(self.torchDevice)).detach().cpu().tolist()
            
            #Clear the ResNet model
            del model_ResNet
            if len(gpus) > 0: torch.cuda.empty_cache()
            
            #Convert list of features to an array
            self.blockFeatures = np.asarray(self.blockFeatures)
            
            #Save features to disk
            np.save(self.dir_features + 'blockFeatures'+suffix, self.blockFeatures)
            
        else: 
            self.blockFeatures = np.load(self.dir_features + 'blockFeatures'+suffix+'.npy', allow_pickle=True)
        
    def computeSalicencyMaps(self, overwrite_saliencyMaps, visualizeSaliencyMaps, suffix=''):
        
        #Extract or load saliency map data for the indicated block files
        if overwrite_saliencyMaps:
            
            #Load pre-trained DenseNet
            model_DenseNet = models.densenet169(weights=weightsDenseNet)

            #Replace the in-built classifier; unclear how this structure and these hyperparameters were determined
            model_DenseNet.classifier = nn.Sequential(
                nn.Linear(model_DenseNet.classifier.in_features, 256),
                nn.ReLU(),
                nn.Dropout(0.4),
                nn.Linear(256, 2),
                nn.LogSoftmax(dim=1)
            )
            model_DenseNet = model_DenseNet.to(self.torchDevice)
            _ = model_DenseNet.train(False)

            #Create GradCAMPlusPlus model; see https://github.com/jacobgil/pytorch-grad-cam for additional models and options
            model_GradCamPlusPlus = GradCAMPlusPlus(model=model_DenseNet, target_layers=[model_DenseNet.features[-1]])

            #Extract features for each batch of sample block images
            #For current GradCamPlusPlus implementation, must manually clear internal copy of the outputs from GPU cache to prevent OOM
            #Do not need to move data to device here, as managed by GradCAMPlusPlus (having already been placed on device)
            self.saliencyMaps = []
            for data in tqdm(self.WSIDataloader, total=self.numWSIData, desc='Computing Saliency Maps', leave=True, ascii=asciiFlag):
                self.saliencyMaps.append(model_GradCamPlusPlus(input_tensor=data, targets=None))
                del model_GradCamPlusPlus.outputs
                if len(gpus) > 0: torch.cuda.empty_cache()
            self.saliencyMaps = np.vstack(self.saliencyMaps)
            
            #Clear DenseNet and GradCamPlusPlus
            del model_DenseNet, model_GradCamPlusPlus
            if len(gpus) > 0: torch.cuda.empty_cache()
            
            #Process each of the resulting maps
            self.blockWeights = []
            for index, saliencyMap in tqdm(enumerate(self.saliencyMaps), total=len(self.saliencyMaps), desc='Processing Saliency Maps', leave=True, ascii=asciiFlag): 
                
                #Load the sample WSI
                imageWSI = cv2.cvtColor(cv2.imread(self.WSIFilenames[index], cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB)
                
                #If visualizations are enabled
                if visualizeSaliencyMaps:
                    
                    #Store the saliency map to disk (keep at original output dimensions)
                    exportImage(self.dir_visuals_saliencyMaps+'saliencyMap_'+self.sampleNames[index]+'.tif', matplotlib.cm.jet(saliencyMap)[:,:,:-1].astype(np.float32))
                
                    #Resize the saliency map to match the WSI dimensions
                    transform = generateTransform(imageWSI.shape[:2], False, False)
                    saliencyMap = transform(np.expand_dims(saliencyMap, 0))[0].numpy()
                    
                    #Overlay the saliency map on the WSI and save it to disk
                    transform = generateTransform([], True, False)
                    overlaid = np.moveaxis(transform(np.moveaxis(imageWSI, -1, 0)).numpy(), 0, -1)
                    if overlayGray: overlaid = np.expand_dims(cv2.cvtColor(overlaid, cv2.COLOR_RGB2GRAY), -1)
                    overlaid = show_cam_on_image(overlaid, saliencyMap, use_rgb=True, colormap=cv2.COLORMAP_HOT, image_weight=1.0-overlayWeight)
                    exportImage(self.dir_visuals_overlaidSaliencyMaps+'overlaidSaliency_'+self.sampleNames[index]+overlayExtension, overlaid)
                
                #Extract saliency map data specific to sample block locations, compute regional importance as the average value, and threshold to get weights
                blockWeights = []
                for locationIndex, locationData in enumerate(self.blockLocations[np.where(self.blockSampleNames == self.sampleNames[index])[0]]):
                    startRow, startColumn = locationData
                    blockSaliencyMap = saliencyMap[startRow:startRow+blockSize, startColumn:startColumn+blockSize]
                    blockImportance = np.mean(blockSaliencyMap)
                    if blockImportance < 0.25: blockWeights.append(0)
                    else: blockWeights.append(blockImportance)
                self.blockWeights += blockWeights
                
            #Convert list of block weights to an array and save to disk
            self.blockWeights = np.asarray(self.blockWeights)
            np.save(self.dir_saliencyMaps + 'blockWeights'+suffix, self.blockWeights)
            
        else:
            self.blockWeights = np.load(self.dir_saliencyMaps + 'blockWeights'+suffix+'.npy', allow_pickle=True)
    
    #Classify extracted block features
    def predict(self, inputs, fusionMode, weights=None):
        
        #Compute the raw block predictions
        predictions = self.model_XGBClassifier.predict(inputs.astype(np.float32))
        
        #If fusion mode is active, multiply the block predictions (using -1 for benign and +1 for malignant) by the matching weights; positive results are malignant
        if fusionMode: 
            predictionsFusion = np.where(predictions==0, -1, 1)*weights
            predictionsFusion = np.where(predictionsFusion>0, 1, 0)
            return predictions.tolist(), predictionsFusion.tolist()
        else: 
            return predictions.tolist()
    
    #Perform cross-validation
    def crossValidation(self):
        
        #If block weights are available, then enable evaluation of fusion mode
        if len(self.blockWeights)>0: fusionMode = True
        else: fusionMode = False
        
        #Allocate samples to folds for cross validation
        if type(manualFolds) != list: folds = [array.tolist() for array in np.array_split(np.random.permutation(self.sampleNames), manualFolds)]
        else: folds = manualFolds
        numFolds = len(folds)

        #Split block features into specified folds, keeping track of originating indices and matched labels
        foldsFeatures, foldsLabels, foldsWeights, foldsBlockSampleNames, foldsBlockNames, foldsBlockLocations, foldsWSILabels = [], [], [], [], [], [], []
        for fold in folds:
            blockIndices = np.concatenate([np.where(self.blockSampleNames == sampleName)[0] for sampleName in fold])
            foldsBlockLocations.append(list(self.blockLocations[blockIndices]))
            foldsFeatures.append(list(self.blockFeatures[blockIndices]))
            foldsLabels.append(list(self.blockLabels[blockIndices]))
            if fusionMode: foldsWeights.append(list(self.blockWeights[blockIndices]))
            foldsBlockSampleNames.append(list(self.blockSampleNames[blockIndices]))
            foldsBlockNames.append(list(self.blockNames[blockIndices]))            
            foldsWSILabels += [self.WSILabels[np.where(self.sampleNames == sampleName)[0]][0] for sampleName in fold]
        
        #Collapse data for later (correct/matched ordered) evaluation of the fold data
        foldsSampleNames = np.asarray(sum(folds, []))
        foldsBlockLocations = np.concatenate(foldsBlockLocations)
        foldsBlockSampleNames = np.concatenate(foldsBlockSampleNames)
        foldsBlockNames = np.concatenate(foldsBlockNames)
        foldsWSILabels = np.asarray(foldsWSILabels)
        foldsWSIFilenames = np.concatenate([self.WSIFilenames[np.where(self.sampleNames == sampleName)[0]] for sampleName in foldsSampleNames])
        
        #Check class distribution between the folds
        #print('B\t M \t Total')
        #for foldNum in range(0, numFolds):
        #    print(np.sum(np.array(foldsLabels[foldNum]) == valueBenign),'\t', np.sum(np.array(foldsLabels[foldNum]) == valueMalignant), '\t', len(foldsLabels[foldNum]))
        
        #Perform training/testing among the folds, testing sequentially (to ensure the correct order) and storing results for later evaluation
        foldsBlockPredictions, foldsBlockPredictionsFusion = [], []
        for foldNum in tqdm(range(0, numFolds), desc='Block Classification', leave=True, ascii=asciiFlag):
            
            #Train on all folds except the one specified
            self.train(np.concatenate(foldsFeatures[:foldNum]+foldsFeatures[foldNum+1:]), np.concatenate(foldsLabels[:foldNum]+foldsLabels[foldNum+1:]))
            
            #Extract the testing data and place on the GPU if able
            dataInput = np.asarray(foldsFeatures[foldNum])
            if len(gpus) > 0: dataInput = cp.asarray(dataInput)
            
            #Classify blocks in the specified, remaining fold
            if fusionMode: 
                predictions, predictionsFusion = self.predict(dataInput, fusionMode, np.asarray(foldsWeights[foldNum]))
                foldsBlockPredictionsFusion += predictionsFusion
            else:
                predictions = self.predict(dataInput, fusionMode)
            foldsBlockPredictions += predictions
            
            #Clear the XGBClassifier model and data on GPU
            del self.model_XGBClassifier, dataInput
            if len(gpus) > 0: 
                torch.cuda.empty_cache() 
                cp._default_memory_pool.free_all_blocks()
        
        #Convert lists of predictions to arrays and collapse labels for evaluation
        foldsLabels = np.concatenate(foldsLabels)
        foldsBlockPredictions = np.asarray(foldsBlockPredictions)
        foldsBlockPredictionsFusion = np.asarray(foldsBlockPredictionsFusion)
        
        #Save results to disk
        dataPrintout, dataPrintoutNames = [foldsBlockNames, foldsLabels, foldsBlockPredictions], ['Names', 'Labels', 'Raw Predictions']
        if fusionMode: 
            dataPrintout.append(foldsBlockPredictionsFusion)
            dataPrintoutNames.append('Fusion Predictions')
        dataPrintout = pd.DataFrame(np.asarray(dataPrintout)).transpose()
        dataPrintout.columns=dataPrintoutNames
        dataPrintout.to_csv(self.dir_results + 'predictions_blocks.csv', index=False)
        
        #Evaluate per-block results
        computeClassificationMetrics(foldsLabels, foldsBlockPredictions, self.dir_results+'results_blocks_initial')
        if fusionMode: computeClassificationMetrics(foldsLabels, foldsBlockPredictionsFusion, self.dir_results+'results_blocks_fusion')
    
        #Classify each WSI, that had its component blocks classified, according to specified threshold of allowable malignant blocks
        foldsSampleLabels, foldsSamplePredictions, foldsSamplePredictionsFusion = [], [], []
        for foldsSampleIndex, sampleName in enumerate(foldsSampleNames):
            blockIndices = np.where(foldsBlockSampleNames == sampleName)[0]
            if thresholdWSI_GT: foldsSampleLabels.append((np.mean(foldsLabels[blockIndices]) >= thresholdWSI)*1)
            foldsSamplePredictions.append((np.mean(foldsBlockPredictions[blockIndices]) >= thresholdWSI)*1)
            if fusionMode: foldsSamplePredictionsFusion.append((np.mean(foldsBlockPredictionsFusion[blockIndices]) >= thresholdWSI)*1)
        
        #Convert list of WSI labels/predictions to arrays
        if thresholdWSI_GT: 
            foldsSampleLabels = np.asarray(foldsSampleLabels)
            mismatchedSamples = foldsSampleNames[np.where(foldsSampleLabels-foldsWSILabels != 0)[0]].tolist()
            if len(mismatchedSamples) > 0: print('\nWarning - Determination of ground-truth labels for WSI using threshold method did not match with actual WSI labels for the following samples:\n' + str(mismatchedSamples)+'\nStrongly advise revising WSI threshold criteria or disabling thresholdWSI_GT.')
        else: foldsSampleLabels = foldsWSILabels
        foldsSamplePredictions = np.asarray(foldsSamplePredictions)
        foldsSamplePredictionsFusion = np.asarray(foldsSamplePredictionsFusion)
        
        #Evaluate per-sample results
        self.evaluateResultsWSI(foldsSampleNames, foldsWSIFilenames, foldsSampleLabels, foldsSamplePredictions, foldsSamplePredictionsFusion, foldsBlockSampleNames, foldsLabels, foldsBlockPredictions, foldsBlockPredictionsFusion, foldsBlockLocations)

    def evaluateResultsWSI(self, sampleNames, WSIFilenames, sampleLabels, samplePredictions, samplePredictionsFusion, blockSampleNames, blockLabels, blockPredictions, blockPredictionsFusion, blockLocations):
        
        #If block weights are available, then enable evaluation of fusion mode
        if len(self.blockWeights)>0: fusionMode = True
        else: fusionMode = False
        
        #Save results to disk
        dataPrintout, dataPrintoutNames = [sampleNames, sampleLabels, samplePredictions], ['Names', 'Labels', 'Raw Predictions']
        if fusionMode: 
            dataPrintout.append(samplePredictionsFusion)
            dataPrintoutNames.append('Fusion Predictions') 
        dataPrintout = pd.DataFrame(np.asarray(dataPrintout)).transpose()
        dataPrintout.columns=dataPrintoutNames
        dataPrintout.to_csv(self.dir_results + 'predictions_WSI.csv', index=False)
        
        #Evaluate original WSI results
        computeClassificationMetrics(sampleLabels, samplePredictions, self.dir_results+'results_WSI_Initial')
        if fusionMode: computeClassificationMetrics(sampleLabels, samplePredictionsFusion, self.dir_results+'results_WSI_Fusion')
        
        #If the labels/predictions should be mapped visually onto the WSI
        if self.visualizeLabelGrids or self.visualizePredictionGrids:
            for sampleIndex, sampleName in tqdm(enumerate(sampleNames), total=len(sampleNames), desc='Visualizing Results', leave=True, ascii=asciiFlag):
                
                #Get indices for the sample blocks
                blockIndices = np.where(blockSampleNames == sampleName)[0]
                
                #Load the sample WSI
                imageWSI = cv2.cvtColor(cv2.imread(WSIFilenames[sampleIndex], cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB)
                if (len(self.cropData) != 0): 
                    cropData, paddingData = self.cropData[sampleIndex], self.paddingData[sampleIndex]
                    imageWSI = np.pad(imageWSI[cropData[0]:cropData[1], cropData[2]:cropData[3]], ((paddingData[0], paddingData[1]), (paddingData[2], paddingData[3]), (0, 0)))
                
                #Create grid overlay objects: valueBenign(other)->green, valueMalignant->red, valueBackground->N/A (background blocks not classified or included)
                if self.visualizeLabelGrids: gridOverlay_GT = np.zeros(imageWSI.shape, dtype=np.uint8)
                gridOverlay_Predictions = np.zeros(imageWSI.shape, dtype=np.uint8)
                gridOverlay_PredictionsFusion = np.zeros(imageWSI.shape, dtype=np.uint8)
                for blockIndex in blockIndices:
                    startRow, startColumn = blockLocations[blockIndex] 
                    posStart, posEnd = (startColumn, startRow), (startColumn+blockSize, startRow+blockSize)
                    if self.visualizeLabelGrids: gridOverlay_GT = rectangle(gridOverlay_GT, posStart, posEnd, (255, 0, 0) if blockLabels[blockIndex] == valueMalignant else (0, 255, 0))
                    gridOverlay_Predictions = rectangle(gridOverlay_Predictions, posStart, posEnd, (255, 0, 0) if blockPredictions[blockIndex] == valueMalignant else (0, 255, 0))
                    gridOverlay_PredictionsFusion = rectangle(gridOverlay_PredictionsFusion, posStart, posEnd, (255, 0, 0) if blockPredictionsFusion[blockIndex] == valueMalignant else (0, 255, 0))
                    
                #Store overlays to disk
                if self.visualizeLabelGrids: exportImage(self.dir_visuals_labelGrids+'overlay_labelGrid_'+sampleName+overlayExtension, gridOverlay_GT)
                exportImage(self.dir_results_labelGrids+'overlay_predictionsGrid_'+sampleName+overlayExtension, gridOverlay_Predictions)
                exportImage(self.dir_results_labelGrids+'overlay_fusionGrid_'+sampleName+overlayExtension, gridOverlay_PredictionsFusion)
                
                #Overlay grids on top of WSI and store to disk
                if self.visualizeLabelGrids: 
                    imageWSI_GT = cv2.addWeighted(imageWSI, 1.0, gridOverlay_GT, overlayWeight, 0.0)
                    exportImage(self.dir_visuals_overlaidLabelGrids+'labelGrid_'+sampleName+overlayExtension, imageWSI_GT)
                imageWSI_Predictions = cv2.addWeighted(imageWSI, 1.0, gridOverlay_Predictions, overlayWeight, 0.0)
                exportImage(self.dir_results_overlaidLabelGrids+'predictionsGrid_'+sampleName+overlayExtension, imageWSI_Predictions)
                imageWSI_PredictionsFusion = cv2.addWeighted(imageWSI, 1.0, gridOverlay_PredictionsFusion, overlayWeight, 0.0)
                exportImage(self.dir_results_overlaidLabelGrids+'fusionGrid_'+sampleName+overlayExtension, imageWSI_PredictionsFusion)
    
    #Train a new XGB Classifier model
    def train(self, inputs, labels):
        self.model_XGBClassifier = XGBClassifier(device=self.device)
        _  = self.model_XGBClassifier.fit(inputs.astype(np.float32), labels)
    
    #Train on all available data and export models
    def exportClassifier(self):
        
        #Setup, train, save, and clear the XGBClassifier model
        self.train(self.blockFeatures, self.blockLabels)
        
        #Save to disk in .json format for easy reloading
        self.model_XGBClassifier.save_model(dir_classifier_models + 'model_XGBClassifier.json')
        
        #Register converter for XGBClassifier
        update_registered_converter(XGBClassifier, "XGBoostXGBClassifier", calculate_linear_classifier_output_shapes, convert_xgboost, options={"nocl": [True, False], "zipmap": [True, False, "columns"]},)
        
        #Convert classifier to onnx format and save to disk
        model_onnx_XGBClassifier = to_onnx(self.model_XGBClassifier, self.blockFeatures.astype(np.float32), target_opset={"": skl2onnx.__max_supported_opset__, "ai.onnx.ml": 3})
        with open(dir_classifier_models + 'model_XGBClassifier.onnx', 'wb') as f:f.write(model_onnx_XGBClassifier.SerializeToString())
        
        del self.model_XGBClassifier, model_onnx_XGBClassifier
        if len(gpus) > 0: 
            torch.cuda.empty_cache() 
            cp._default_memory_pool.free_all_blocks()
            
        #Still need to export ResNet50 model to onnx as well for integration (will be leaving DenseNet and GradCam++ (decision fusion) aside, as they do not appear to impact the results)
    
    #Load a pretrained model
    def loadClassifier(self):
        self.model_XGBClassifier = XGBClassifier()
        self.model_XGBClassifier.load_model(dir_classifier_models + 'model_XGBClassifier.json')
        self.model_XGBClassifier._Booster.set_param({'device': self.device})
        
        #Using ONNX; can be done, but GPU handling here would need work to perform correctly/well in Python
        #self.model_XGBClassifier = onnxruntime.InferenceSession(dir_classifier_models + 'model_XGBClassifier.onnx', providers = [('CUDAExecutionProvider', {"device_id": gpus[-1]}), 'CPUExecutionProvider'])
        #if len(gpus) > 0: self.model_XGBClassifier.predict = lambda inputs: session.run(None, {'X': cp.asarray(inputs)})[0]
        #else: self.model_XGBClassifier.predict = lambda inputs: session.run(None, {'X': inputs})[0]
        
    #Classify a sample WSI
    def classifyWSI(self, evaluatePredictions):
        
        #Place data on the GPU if able
        dataInput = np.asarray(self.blockFeatures.astype(np.float32))
        if len(gpus) > 0: dataInput = cp.asarray(dataInput)
        
        #Classify blocks
        if fusionMode_WSI: blockPredictions, blockPredictionsFusion = self.predict(dataInput, fusionMode_WSI, self.blockWeights)
        else: blockPredictions, blockPredictionsFusion = self.predict(dataInput, fusionMode_WSI), []
        blockPredictions, blockPredictionsFusion = np.asarray(blockPredictions), np.asarray(blockPredictionsFusion)
        
        #Clear the XGBClassifier model and data on GPU
        del self.model_XGBClassifier, dataInput
        if len(gpus) > 0: 
            torch.cuda.empty_cache() 
            cp._default_memory_pool.free_all_blocks()
        
        #Classify each WSI, that had its component blocks classified, according to specified threshold of allowable malignant blocks
        sampleLabels, samplePredictions, samplePredictionsFusion, sampleBlockIndices = [], [], [], []
        for sampleIndex, sampleName in enumerate(self.sampleNames):
            blockIndices = np.where(self.blockSampleNames == sampleName)[0]
            sampleBlockIndices.append(blockIndices)
            samplePredictions.append((np.mean(blockPredictions[blockIndices]) >= thresholdWSI)*1)
            if fusionMode_WSI: samplePredictionsFusion.append((np.mean(blockPredictionsFusion[blockIndices]) >= thresholdWSI)*1)
        
        #Convert list of WSI labels/predictions to arrays
        samplePredictions = np.asarray(samplePredictions)
        samplePredictionsFusion = np.asarray(samplePredictionsFusion)
        
        #Evaluate per-sample results
        if evaluatePredictions: self.evaluateResultsWSI(self.sampleNames, self.WSIFilenames, self.WSILabels, samplePredictions, samplePredictionsFusion, self.blockSampleNames, None, blockPredictions, blockPredictionsFusion, self.blockLocations)
        
        #Create prediction arrays for reconstruction model input data and save them to disk
        if classifierRecon:
            predictionMaps, predictionsFusionMaps = [], []
            for sampleIndex, sampleName in tqdm(enumerate(self.sampleNames), total=len(self.sampleNames), desc='Recon Data', leave=True, ascii=asciiFlag):
                blockIndices = sampleBlockIndices[sampleIndex]
                predictionLocations = self.blockLocations[blockIndices]//blockSize
                predictionMap = np.full((self.shapeData[sampleIndex]), valueBackground)
                predictionMap[predictionLocations[:,0], predictionLocations[:,1]] = blockPredictions[blockIndices]
                predictionMaps.append(predictionMap)
                if visualizeInputData_recon: exportImage(dir_recon_visuals_inputData+'predictionMap_'+sampleName+'.tif', cmapClasses(predictionMap)[:,:,:3].astype(np.uint8)*255)
                if fusionMode_WSI:
                    predictionFusionMap = np.full((self.shapeData[sampleIndex]), valueBackground)
                    predictionFusionMap[predictionLocations[:,0], predictionLocations[:,1]] = blockPredictionsFusion[blockIndices]
                    predictionsFusionMaps.append(predictionFusionMap)
                    if visualizeInputData_recon: exportImage(dir_recon_visuals_inputData+'fusionMap_'+sampleName+'.tif', cmapClasses(predictionFusionMap)[:,:,:3].astype(np.uint8)*255)
            predictionMaps = np.asarray(predictionMaps, dtype='object')
            np.save(dir_recon_inputData + 'predictionMaps', predictionMaps)
            if fusionMode_WSI: 
                predictionsFusionMaps = np.asarray(predictionsFusionMaps, dtype='object')
                np.save(dir_recon_inputData + 'fusionMaps', predictionsFusionMaps)
                

In [8]:
#==================================================================
#DATA_PROCESSING
#==================================================================

#Compute the ratio of minimum foreground to block area required for a block to be considered as holding foreground data
blockBackgroundRatio = minimumForegroundArea/((blockSize*cameraResolution)**2)
if blockBackgroundRatio > 1.0: sys.exit('\nError - Minimum foreground area specified for foreground data exceeds the given block size.')

#Define general labels and values to use
labelBenign, labelMalignant, labelExclude = '0', '1', '2'
valueBenign, valueMalignant, valueBackground = int(labelBenign), int(labelMalignant), 2
cmapClasses = colors.ListedColormap(['lime', 'red', 'black'])

#Determine overlay and grid image export extension; exporting as .tif for losselss and .jpg for compressed
if overlayLossless: overlayExtension = '.tif'
else: overlayExtension = '.jpg'

#Determine offset to avoid overlapping squares in grid visualization
gridThicknessOffset = gridThickness//2

#Load sample metadata for blocks and WSI data; keep seperate until working with reconstruction model input data to prevent accidental evaluation of classifier model with training data
try:
    sampleNames_blocks, WSILabels_blocks = loadMetadata_WSI(file_blocks_metadataWSI)
    WSILabels_blocks = WSILabels_blocks.astype(int)
    WSIFilenames_blocks = np.asarray([dir_blocks_inputWSI + sampleName + '.jpg' for sampleName in sampleNames_blocks])
except:
    print('\nWarning - There does not appear to be any metadata available for block samples.')
    sampleNames_blocks, WSILabels_blocks, WSIFilenames_blocks = np.asarray([]), np.asarray([]), np.asarray([])
try:
    sampleNames_WSI, WSILabels_WSI = loadMetadata_WSI(file_WSI_metadataWSI)
    WSILabels_WSI = WSILabels_WSI.astype(int)
    WSIFilenames_WSI = np.asarray([dir_WSI_inputs + sampleName + '.jpg' for sampleName in sampleNames_WSI])
except:
    print('\nWarning - There does not appear to be any metadata available for WSI samples.')
    sampleNames_WSI, WSILabels_WSI, WSIFilenames_WSI = np.asarray([]), np.asarray([]), np.asarray([])

#Combine sample names and WSI filenames for classifierRecon and/or blockBackgroundValue determination for classifierWSI
sampleNames_recon, WSIFilenames_recon = np.concatenate([sampleNames_blocks, sampleNames_WSI]), np.concatenate([WSIFilenames_blocks, WSIFilenames_WSI])

#If configured for block classification and/or model export
if classifierBlocks or classifierExport: 
    
    #Load and process metadata for available blocks and their originating WSI
    blockSampleNames_blocks, indices_blocks, locations_blocks, blockLabels_blocks = loadMetadata_blocks(file_blocks_metadataBlocks)
    blockLabels_blocks = blockLabels_blocks.astype(int)
    blockNames_blocks = np.asarray([blockSampleNames_blocks[index] + '_' + indices_blocks[index] for index in range(0, len(blockSampleNames_blocks))])
    blockFilenames_blocks = np.asarray([dir_blocks_inputBlocks + blockSampleNames_blocks[index] + os.path.sep + 'PS'+blockSampleNames_blocks[index]+'_'+str(indices_blocks[index])+'_'+str(locations_blocks[index, 0])+'_'+str(locations_blocks[index, 1])+'.tif' for index in range(0, len(blockSampleNames_blocks))])
    
    #Prepare classifier
    modelClassifier_blocks = Classifier('blocks', blockNames_blocks, blockFilenames_blocks, blockSampleNames_blocks, locations_blocks, sampleNames_blocks, WSIFilenames_blocks, WSILabels_blocks, blockLabels=blockLabels_blocks)
    
    #Compute block features
    modelClassifier_blocks.computeFeatures(overwrite_blocks_features)
    
    #Compute saliency maps for the blocks
    if fusionMode_blocks: modelClassifier_blocks.computeSalicencyMaps(overwrite_blocks_saliencyMaps, visualizeSaliencyMaps_blocks)
    
    #Perform cross-validation
    if classifierBlocks: modelClassifier_blocks.crossValidation()
    
    #Export classifier components
    if classifierExport: modelClassifier_blocks.exportClassifier()
        
    #Clean RAM of larger object(s)
    del modelClassifier_blocks
    cleanup()

#If classification of whole WSI or generation of reconstruction model input data should be performed
if classifierWSI or classifierRecon: 
    
    #If a background value for thresholding block data has not been set, determine one across all available WSI
    if blockBackgroundValue == -1:
        otsuThresholds = np.asarray([cv2.threshold(cv2.cvtColor(cv2.imread(filename, cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2GRAY),0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[0] for filename in tqdm(WSIFilenames_recon, desc='Background Value Determination', leave=True, ascii=asciiFlag)])
        blockBackgroundValue = int(otsuThresholds.min())
        print('For available WSI, the recommended value for blockBackgroundValue is: '+str(blockBackgroundValue))
        
    #If the overwrite is enabled, split WSI (excluding any that were used in training the classifier) into blocks and save to disk
    if overwrite_WSI_blocks: 
        blockNames_WSI_WSI, blockFilenames_WSI_WSI, blockSampleNames_WSI_WSI, blockLocations_WSI_WSI, cropData_WSI_WSI, paddingData_WSI_WSI, shapeData_WSI_WSI = extractBlocks(WSIFilenames_WSI)
        blockData_WSI = np.concatenate([np.expand_dims(blockNames_WSI_WSI, -1), np.expand_dims(blockFilenames_WSI_WSI, -1), np.expand_dims(blockSampleNames_WSI_WSI, -1), blockLocations_WSI_WSI], 1)
        np.save(dir_WSI_blocks + 'blockData_WSI_WSI', blockData_WSI)
        WSIData_WSI = np.concatenate([cropData_WSI_WSI, paddingData_WSI_WSI, shapeData_WSI_WSI], 1)
        np.save(dir_WSI_blocks + 'WSIData_WSI_WSI', WSIData_WSI)
    else: 
        blockData_WSI = np.load(dir_WSI_blocks + 'blockData_WSI_WSI.npy', allow_pickle=True)
        blockNames_WSI_WSI, blockFilenames_WSI_WSI, blockSampleNames_WSI_WSI, blockLocations_WSI_WSI = np.split(blockData_WSI, [1, 2, 3], 1)
        blockLocations_WSI_WSI = blockLocations_WSI_WSI.astype(int)
        WSIData_WSI = np.load(dir_WSI_blocks + 'WSIData_WSI_WSI.npy', allow_pickle=True)
        cropData_WSI_WSI, paddingData_WSI_WSI, shapeData_WSI_WSI = np.split(WSIData_WSI, [4, 8], 1)
    
    #Prepare classifier; loading the pre-trained model
    modelClassifier_WSI = Classifier('WSI', blockNames_WSI_WSI, blockFilenames_WSI_WSI, blockSampleNames_WSI_WSI, blockLocations_WSI_WSI, sampleNames_WSI, WSIFilenames_WSI, WSILabels_WSI, cropData=cropData_WSI_WSI, paddingData=paddingData_WSI_WSI, shapeData=shapeData_WSI_WSI)
    modelClassifier_WSI.loadClassifier()
    
    #Compute block features
    modelClassifier_WSI.computeFeatures(overwrite_WSI_features, '_WSI_WSI')
    
    #Compute saliency maps for the blocks
    if fusionMode_WSI: modelClassifier_WSI.computeSalicencyMaps(overwrite_WSI_saliencyMaps, visualizeSaliencyMaps_WSI, '_WSI_WSI')
    
    #Classify the WSI and evaluate results
    if classifierWSI: modelClassifier_WSI.classifyWSI(True)
    
    #Clean RAM of larger object(s)
    del modelClassifier_WSI
    cleanup()

#If data should be generated for reconstruction model, extract blocks/data for WSI used to train the classifier, merge with those not used, and export for later use
if classifierRecon:
    
    #Either split WSI (excluding any that were used in training the classifier) into blocks and save to disk, or load data for those previously generated 
    if overwrite_WSI_blocks: 
        blockNames_WSI_blocks, blockFilenames_WSI_blocks, blockSampleNames_WSI_blocks, blockLocations_WSI_blocks, cropData_WSI_blocks, paddingData_WSI_blocks, shapeData_WSI_blocks = extractBlocks(WSIFilenames_blocks)
        blockData_blocks = np.concatenate([np.expand_dims(blockNames_WSI_blocks, -1), np.expand_dims(blockFilenames_WSI_blocks, -1), np.expand_dims(blockSampleNames_WSI_blocks, -1), blockLocations_WSI_blocks], 1)
        np.save(dir_WSI_blocks + 'blockData_WSI_blocks', blockData_blocks)
        WSIData_blocks = np.concatenate([cropData_WSI_blocks, paddingData_WSI_blocks, shapeData_WSI_blocks], 1)
        np.save(dir_WSI_blocks + 'WSIData_WSI_blocks', WSIData_blocks)
        
    else: 
        blockData_blocks = np.load(dir_WSI_blocks + 'blockData_WSI_blocks.npy', allow_pickle=True)
        blockNames_WSI_blocks, blockFilenames_WSI_blocks, blockSampleNames_WSI_blocks, blockLocations_WSI_blocks = np.split(blockData_blocks, [1, 2, 3], 1)
        blockLocations_WSI_blocks = blockLocations_WSI_blocks.astype(int)
        WSIData_blocks = np.load(dir_WSI_blocks + 'WSIData_WSI_blocks.npy', allow_pickle=True)
        cropData_WSI_blocks, paddingData_WSI_blocks, shapeData_WSI_blocks = np.split(WSIData_blocks, [4, 8], 1)
    
    #Prepare classifier
    modelClassifier_WSI_blocks = Classifier('WSI', blockNames_WSI_blocks, blockFilenames_WSI_blocks, blockSampleNames_WSI_blocks, blockLocations_WSI_blocks, sampleNames_blocks, WSIFilenames_blocks, WSILabels_blocks, cropData=cropData_WSI_blocks, paddingData=paddingData_WSI_blocks, shapeData=shapeData_WSI_blocks)
    
    #Compute block features
    modelClassifier_WSI_blocks.computeFeatures(overwrite_WSI_features, '_WSI_blocks')
    
    #Compute saliency maps for the blocks
    if fusionMode_WSI: modelClassifier_WSI_blocks.computeSalicencyMaps(overwrite_WSI_saliencyMaps, visualizeSaliencyMaps_WSI, '_WSI_blocks')
    
    #Clean RAM of larger object(s)
    del modelClassifier_WSI_blocks
    cleanup()
    
    #Merge metadata for all WSI and store consolidated data to disk
    WSILabels_recon = np.concatenate([WSILabels_blocks, WSILabels_WSI])
    cropData_recon = np.concatenate([cropData_WSI_blocks, cropData_WSI_WSI])
    paddingData_recon = np.concatenate([paddingData_WSI_blocks, paddingData_WSI_WSI])
    shapeData_recon = np.concatenate([shapeData_WSI_blocks, shapeData_WSI_WSI])
    WSIData_recon = np.concatenate([np.expand_dims(sampleNames_recon, -1), np.expand_dims(WSIFilenames_recon, -1), np.expand_dims(WSILabels_recon, -1), cropData_recon, paddingData_recon, shapeData_recon], 1)
    np.save(dir_recon_inputData + 'WSIData_recon', WSIData_blocks)

    #Merge metadata for all blocks and store consolidated data to disk
    blockNames_recon = np.concatenate([blockNames_WSI_blocks, blockNames_WSI_WSI])
    blockFilenames_recon = np.concatenate([blockFilenames_WSI_blocks, blockFilenames_WSI_WSI])
    blockSampleNames_recon = np.concatenate([blockSampleNames_WSI_blocks, blockSampleNames_WSI_WSI])
    blockLocations_recon = np.concatenate([blockLocations_WSI_blocks, blockLocations_WSI_WSI], dtype=np.int64) 
    blockData_recon = np.concatenate([np.atleast_2d(blockNames_recon.T).T, np.atleast_2d(blockFilenames_recon.T).T,  np.atleast_2d(blockSampleNames_recon.T).T, blockLocations_recon], 1)
    np.save(dir_recon_inputData + 'blockData_recon', blockData_blocks)
    
    #Prepare classifier, loading the pre-trained model
    modelClassifier_recon = Classifier('recon', blockNames_recon, blockFilenames_recon, blockSampleNames_recon, blockLocations_recon, sampleNames_recon, WSIFilenames_recon, WSILabels_recon, cropData_recon, paddingData_recon, shapeData_recon)
    modelClassifier_recon.loadClassifier()
    
    #Classify the WSI; do not evaluate as they include WSI data used in training the classifier in the first place
    modelClassifier_recon.classifyWSI(False)
    
    #Clean RAM of larger object(s)
    del modelClassifier_recon
    cleanup()

    

Feature Extraction:   0%|          | 0/1129 [00:00<?, ?it/s]

Computing Saliency Maps:   0%|          | 0/3 [00:00<?, ?it/s]

Processing Saliency Maps:   0%|          | 0/66 [00:00<?, ?it/s]

Block Classification:   0%|          | 0/5 [00:00<?, ?it/s]

Visualizing Results:   0%|          | 0/60 [00:00<?, ?it/s]

...

Feature Extraction:   0%|          | 0/1506 [00:00<?, ?it/s]

Computing Saliency Maps:   0%|          | 0/3 [00:00<?, ?it/s]

Processing Saliency Maps:   0%|          | 0/66 [00:00<?, ?it/s]

RANDS Data:   0%|          | 0/281 [00:00<?, ?it/s]

NameError: name 'modelClassifier_WSI_RANDS' is not defined